In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/path/to/CameraTraps')  # CameraTraps repository base dir

In [ ]:
import copy
import os
from datetime import datetime
import json

from azure.cosmos.cosmos_client import CosmosClient
from azure.storage.blob import generate_container_sas, ContainerSasPermissions

from data_management.megadb.megadb_utils import MegadbUtils

# Update the SAS keys in the `datasets` table of MegaDB

Credentials for the MegaDB, `COSMOS_ENDPOINT` and `COSMOS_KEY`, need to be environment variables. 

In [ ]:
# Initialize Cosmos DB client
url = os.environ['COSMOS_ENDPOINT']
key = os.environ['COSMOS_WRITE_KEY']
client = CosmosClient(url, credential=key)

database = client.get_database_client('camera-trap')
container_datasets = database.get_container_client('datasets')

# Get the datasets table
megadb_utils = MegadbUtils(url=url, key=key)
datasets = megadb_utils.get_datasets_table()

In [ ]:
# here are the list of storage accounts the datasets live under - should only be a handful

all_sa = set()
for dataset_name, d in datasets.items():
    all_sa.add(d['storage_account'])

all_sa

In [ ]:
# fill out this dict with storage account name : key

sa_keys = {
    'lilablobssc': ''
}

In [ ]:
# set an expiry date for the SAS signatures
expiry_date = datetime(2026, 1, 1)

new_datasets = []

for dataset_name, d in datasets.items():
    d_new = copy.deepcopy(d)
    
    key_no_query = generate_container_sas(
        d['storage_account'], d['container'], 
        account_key=sa_keys[d['storage_account']],
        permission=ContainerSasPermissions(read=True, list=True),
        expiry=expiry_date
    )
    d_new['container_sas_key'] = '?' + key_no_query
    new_datasets.append(d_new)

In [ ]:
# update the entries (the "id" field will be used to match the new entries to existing ones and update them)

for d in new_datasets:
    res = container_datasets.upsert_item(d)